    16_230109_100_kyungki_apt_김포
   
     
     data : /aiffel/aiffel/000_aiffelthon_230111/3김연수/002_data_dir/002_apt_data/apt_김포_동두천_808.xls

In [1]:
#1. package load
import requests
import json
import os
import math
import time
import pandas as pd
import numpy as np
import geopandas as gpd
import folium
import matplotlib.pyplot as plt

from shapely.geometry import mapping, shape, Point, Polygon, LineString
from folium import plugins
from folium.plugins import MarkerCluster, HeatMap
print('슝~')

슝~


In [2]:
#2. data load  ...SGIS data 가져오기 # SGIS API 사이트의 본인의 consumer_key, consumer_secret를 입력합니다.
consumer_key = '2f74504a63224ee09f2b'       # 서비스ID
consumer_secret =  '63a0ffef893e43a48ae2' # 보안Key


In [3]:
#2. data load  ... SGIS data 가져오기 # SGIS ACCESS TOKEN을 리턴 해주는 함수
def get_access_token(consumer_key, consumer_secret) -> None:
    temp_res = requests.get(
        url="https://sgisapi.kostat.go.kr/OpenAPI3/auth/authentication.json",
        params={
            "consumer_key": consumer_key,
            "consumer_secret": consumer_secret
        }
    )

    access_token = temp_res.json()["result"]["accessToken"]
   
    return access_token
print('슝~')

슝~


In [4]:
#9 Folium를 활용한 공간 데이터 시각화
### 전국도시철도역사정보표준데이터 가져오기

apt_df = pd.read_excel("/aiffel/aiffel/000_aiffelthon_230111/3김연수/002_data_dir/002_apt_data/apt_김포_동두천_808.xls")
apt_df.head()

,시군명,공동주택구분,공동주택명정보,읍면동주소,지번주소,사용검사일자,층수,동수,세대수,의무관리대상여부
0,김포시,NaN,E편한세상 김포로얄하임,통진읍,513,2021-06-08,18,7,574,Y
1,김포시,NaN,GS 자이아파트,풍무동,766,2010-06-17,18,8,373,Y
2,김포시,NaN,GS 자이아파트,풍무동,764,2010-06-17,19,8,445,Y
3,김포시,NaN,LH한강센트럴리버,구래동,NaN,2015-06-29,29,8,820,N
4,김포시,NaN,LH호수마을5단지,구래동,1817327,2016-04-21,25,8,890,N


In [5]:
apt_df.drop(["공동주택구분"], axis=1,inplace=True)
apt_df.head()

,시군명,공동주택명정보,읍면동주소,지번주소,사용검사일자,층수,동수,세대수,의무관리대상여부
0,김포시,E편한세상 김포로얄하임,통진읍,513,2021-06-08,18,7,574,Y
1,김포시,GS 자이아파트,풍무동,766,2010-06-17,18,8,373,Y
2,김포시,GS 자이아파트,풍무동,764,2010-06-17,19,8,445,Y
3,김포시,LH한강센트럴리버,구래동,NaN,2015-06-29,29,8,820,N
4,김포시,LH호수마을5단지,구래동,1817327,2016-04-21,25,8,890,N


In [6]:
apt_df['지번주소'] = apt_df['지번주소'].astype(str)

In [7]:
#okok
apt_df['addr'] = '경기도 ' + apt_df[['시군명', '읍면동주소','지번주소']].agg(' '.join, axis=1)
apt_df.head()

,시군명,공동주택명정보,읍면동주소,지번주소,사용검사일자,층수,동수,세대수,의무관리대상여부,addr
0,김포시,E편한세상 김포로얄하임,통진읍,513,2021-06-08,18,7,574,Y,경기도 김포시 통진읍 513
1,김포시,GS 자이아파트,풍무동,766,2010-06-17,18,8,373,Y,경기도 김포시 풍무동 766
2,김포시,GS 자이아파트,풍무동,764,2010-06-17,19,8,445,Y,경기도 김포시 풍무동 764
3,김포시,LH한강센트럴리버,구래동,nan,2015-06-29,29,8,820,N,경기도 김포시 구래동 nan
4,김포시,LH호수마을5단지,구래동,1817327,2016-04-21,25,8,890,N,경기도 김포시 구래동 1817327


In [8]:
# https://sgisapi.kostat.go.kr/OpenAPI3/addr/stage.json

In [9]:
#5 seoul university   # 주소를 입력해 주소의 위도, 경도를 가지고 오는 함수
def get_adress_x_y(address) -> None:
    # SGIS access_token 가져오기
    access_token = get_access_token(consumer_key , consumer_secret)

    url="https://sgisapi.kostat.go.kr/OpenAPI3/addr/geocode.json"
    params={
        "accessToken": access_token,
        "address": address
    }

    temp_res = requests.get(
        url=url,
        params=params
    ).json()["result"]["resultdata"]

    return temp_res[0]["x"], temp_res[0]["y"]

print('슝~')

슝~


In [10]:
#5 seoul university .. 주소 기반 위도,경도 추가
cnt = 0
for _, row in apt_df.iterrows():
    try:
        x, y = get_adress_x_y(row["addr"])
     
        apt_df.loc[_, "x"] = x
        apt_df.loc[_, "y"] = y
    except:
        print(row["addr"] + '은(는) 검색에 실패했습니다')
        cnt = cnt + 1

print("실패건수 : ", cnt)
apt_df.head()


경기도 김포시 통진읍 513은(는) 검색에 실패했습니다
경기도 김포시 구래동 1817327은(는) 검색에 실패했습니다
경기도 김포시 고촌읍 1110은(는) 검색에 실패했습니다
경기도 김포시 고촌읍 1102은(는) 검색에 실패했습니다
경기도 김포시 양촌읍 1316은(는) 검색에 실패했습니다
경기도 김포시 양촌읍 1318은(는) 검색에 실패했습니다
경기도 김포시 양촌읍 1332은(는) 검색에 실패했습니다
경기도 김포시 고촌읍 539-1은(는) 검색에 실패했습니다
경기도 김포시 고촌읍 469-3은(는) 검색에 실패했습니다
경기도 김포시 고촌읍 1079은(는) 검색에 실패했습니다
경기도 김포시 양촌읍 1249은(는) 검색에 실패했습니다
경기도 김포시 양촌읍 1247은(는) 검색에 실패했습니다
경기도 김포시 양촌읍 1238은(는) 검색에 실패했습니다
경기도 김포시 양촌읍 1235은(는) 검색에 실패했습니다
경기도 김포시 양촌읍 1254은(는) 검색에 실패했습니다
경기도 김포시 양촌읍 1268은(는) 검색에 실패했습니다
경기도 김포시 양촌읍 1241은(는) 검색에 실패했습니다
경기도 김포시 통진읍 44816은(는) 검색에 실패했습니다
경기도 김포시 구래동 1824479은(는) 검색에 실패했습니다
경기도 김포시 구래동 1816201은(는) 검색에 실패했습니다
경기도 김포시 양촌읍 1551은(는) 검색에 실패했습니다
경기도 김포시 장기동 83277은(는) 검색에 실패했습니다
경기도 김포시 장기동 58472은(는) 검색에 실패했습니다
경기도 김포시 구래동 1823690은(는) 검색에 실패했습니다
경기도 김포시 구래동 1821863은(는) 검색에 실패했습니다
경기도 김포시 통진읍 445-1은(는) 검색에 실패했습니다
경기도 김포시 고촌읍 1055은(는) 검색에 실패했습니다
경기도 김포시 통진읍 498은(는) 검색에 실패했습니다
경기도 김포시 통진읍 512은(는) 검색에 실패했습니다
경기도 김포시 구래동 1821192은(는) 검색에 실패했습니다
경기도 김포시 구래동 1821984

,시군명,공동주택명정보,읍면동주소,지번주소,사용검사일자,층수,동수,세대수,의무관리대상여부,addr,x,y
0,김포시,E편한세상 김포로얄하임,통진읍,513,2021-06-08,18,7,574,Y,경기도 김포시 통진읍 513,NaN,NaN
1,김포시,GS 자이아파트,풍무동,766,2010-06-17,18,8,373,Y,경기도 김포시 풍무동 766,931838.699171291,1956271.7568607
2,김포시,GS 자이아파트,풍무동,764,2010-06-17,19,8,445,Y,경기도 김포시 풍무동 764,931579.247791759,1956311.18697879
3,김포시,LH한강센트럴리버,구래동,nan,2015-06-29,29,8,820,N,경기도 김포시 구래동 nan,922861.700741376029,1960960.01359999995
4,김포시,LH호수마을5단지,구래동,1817327,2016-04-21,25,8,890,N,경기도 김포시 구래동 1817327,NaN,NaN


In [11]:
#5 seoul university  ..# SGIS 좌표변환 API를 활용한 좌표변환 함수
def get_transcoord(src, dst, posX, posY) -> None:
    access_token = get_access_token(consumer_key, consumer_secret)
    url="https://sgisapi.kostat.go.kr/OpenAPI3/transformation/transcoord.json"
    params={
        "accessToken": access_token,
        "src": src, # 현재 좌표체계
        "dst": dst, # 변환할 좌표체계
        "posX": posX, # 변환할 좌표 X값
        "posY":	posY # 변환할 좌표 Y값
    }

    temp_dict = requests.get(
        url=url,
        params=params
    ).json()["result"]

    return temp_dict["posX"], temp_dict["posY"] # 변환된 좌표값 딕셔너리의 value 리턴

print('슝~')


슝~


In [12]:
#5 seoul university  .. 좌표계가 UTM-K이므로 WSG84좌표계로 변환
for _, row in apt_df.iterrows():
    try:
        x, y = get_transcoord("EPSG:5179", "EPSG:4326", row["x"], row["y"])
    
        apt_df.loc[_, "lng"] = x
        apt_df.loc[_, "lat"] = y

    except:
        pass
apt_df.head()

,시군명,공동주택명정보,읍면동주소,지번주소,사용검사일자,층수,동수,세대수,의무관리대상여부,addr,x,y,lng,lat
0,김포시,E편한세상 김포로얄하임,통진읍,513,2021-06-08,18,7,574,Y,경기도 김포시 통진읍 513,NaN,NaN,NaN,NaN
1,김포시,GS 자이아파트,풍무동,766,2010-06-17,18,8,373,Y,경기도 김포시 풍무동 766,931838.699171291,1956271.7568607,126.727799,37.603342
2,김포시,GS 자이아파트,풍무동,764,2010-06-17,19,8,445,Y,경기도 김포시 풍무동 764,931579.247791759,1956311.18697879,126.724856,37.603678
3,김포시,LH한강센트럴리버,구래동,nan,2015-06-29,29,8,820,N,경기도 김포시 구래동 nan,922861.700741376029,1960960.01359999995,126.625613,37.644886
4,김포시,LH호수마을5단지,구래동,1817327,2016-04-21,25,8,890,N,경기도 김포시 구래동 1817327,NaN,NaN,NaN,NaN


In [13]:
#5 seoul university  .. 시각화 .. 초기 중심값 좌표가 들어가야 합니다.

# 서울시 중심부 위도, 경도
center = [37.541, 126.986]

# map이 서울 중심부에 위치하고, zoom 레벨은 12로 시작합니다.
map = folium.Map(
    location=center,
    zoom_start=12

)

print('슝~')

슝~


In [14]:
# #5 seoul university # 대학 이름, 위도, 경도를 가져와 시각화
# for _, row in apt_df.iterrows():
#     try:
#         folium.Marker(
#             location=[row["lat"], row["lng"]], # 좌표
#             popup=row["공동주택명정보"], # 클릭시 표시될 popup 내용
#             icon=folium.Icon(color="cadetblue", icon="info-sign") # 표시될 아이콘 설정
#         ).add_to(map)
#     except:
#         pass

# print('슝~')

슝~


In [19]:
#2. 프로젝트(1)  # MarkerCluster 객체 
marker_cluster = MarkerCluster().add_to(map)

#maker 객체 추가
for idx, row in apt_df.iterrows():
    try:
        folium.Marker(
            location=[apt_df.loc[idx, "lat"], apt_df.loc[idx, "lng"]],
            popup=apt_df.loc[idx, "공동주택명정보"],
            icon=folium.Icon(color="cadetblue", icon="house")            
        ).add_to(marker_cluster)
    except:
        pass

print('슝~')

슝~


In [20]:
#5 seoul university 
map

In [16]:
map.save("/aiffel/aiffel/000_aiffelthon_230111/3김연수/003_map_xml/apt_gimpo.html")

In [17]:
apt_df.to_csv("apt_gimpo.csv", index = False)